# Исследование надежности заемщиков


In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

# Шаг 3. Исследуем данные и ответьте на вопросы

3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# Создаём сводную таблицу
pivot = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum'])
pivot

,count,sum
,debt,debt
children,,
0,14091,1063
1,4808,444
2,2052,194
3,330,27
4,41,4
5,9,0


In [23]:
# Считаем конверсию и сортируем таблицу по её убыванию
pivot['conversion'] = pivot[('sum', 'debt')] / pivot[('count', 'debt')]
pivot.sort_values(by='conversion', ascending=False, inplace=True)
pivot

,count,sum,conversion
,debt,debt,
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4808,444,0.092346
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


**Вывод:** зависимость есть. В данном датасете, посчитав конверсию, мы можем сказать, что плательщики, у которых есть дети, имеют больший шанс не вернуть деньги вовремя
аномальной конверсией здесь считаются клиенты, имеющие 5 детей: в данном случае их всего 9 и они вовремя возвращали кредит.

# 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
# Создаём сводную таблицу
pivot = data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum'])
pivot

,count,sum
,debt,debt
family_status,,
Не женат / не замужем,2796,273
в разводе,1189,84
вдовец / вдова,951,63
гражданский брак,4134,385
женат / замужем,12261,927


In [25]:
# Считаем конверсию и сортируем таблицу по её убыванию
pivot['conversion'] = pivot[('sum', 'debt')] / pivot[('count', 'debt')]
pivot.sort_values(by='conversion', ascending=False, inplace=True)
pivot

,count,sum,conversion
,debt,debt,
family_status,,,
Не женат / не замужем,2796,273,0.097639
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246


**Вывод:** зависимость есть. В этом случае, посчитав конверсию, мы можем сказать, что незамужние (неженатые) плательщики имеют меньший шанс вернуть деньги вовремя

# 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
# Создаём сводную таблицу
pivot = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum'])
pivot

,count,sum
,debt,debt
total_income_category,,
A,25,2
B,5014,354
C,15921,1353
D,349,21
E,22,2


In [27]:
# Считаем конверсию и сортируем таблицу по её убыванию
pivot['conversion'] = pivot[('sum', 'debt')] / pivot[('count', 'debt')]
pivot.sort_values(by='conversion', ascending=False, inplace=True)
pivot

,count,sum,conversion
,debt,debt,
total_income_category,,,
E,22,2,0.090909
C,15921,1353,0.084982
A,25,2,0.080000
B,5014,354,0.070602
D,349,21,0.060172


**Вывод:** здесь, посчитав конверсию по уровню дохода, можно сказать, что плательщики с наименьшим доходом (класс Е) имеют наибольший шанс просрочить платёж; заемщики с более высоким уровнем дохода имеют более высокую вероятность выплатить кредит в срок, чем те, у кого доход ниже.

# 3.4 Как разные цели кредита влияют на его возврат в срок?

In [28]:
# Создаём сводную таблицу
pivot = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])
pivot

,count,sum
,debt,debt
purpose_category,,
операции с автомобилем,4279,400
операции с недвижимостью,10751,780
получение образования,3988,369
проведение свадьбы,2313,183


In [29]:
# Считаем конверсию и сортируем таблицу по её убыванию
pivot['conversion'] = pivot[('sum', 'debt')] / pivot[('count', 'debt')]
pivot.sort_values(by='conversion', ascending=False, inplace=True)
pivot

,count,sum,conversion
,debt,debt,
purpose_category,,,
операции с автомобилем,4279,400,0.093480
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118
операции с недвижимостью,10751,780,0.072551


**Вывод:** По таблице мы можем сделать вывод, что, например, кредит на образование и операции с автомобилем вовремя выплачивается реже, чем кредит, взятый на проведение свадьбы или на операции с недвижимостью

# 3.5 Возможные причины появления пропусков в исходных данных.

**Ответ:** 
1. Пользователь недозаполнил форму на сайте
2. Сбои в системе
3. Необязательные поля
4. Данные были случайно удалены или не получены

# 3.6 Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных?

**Ответ:** Медиана менее подвержена выбросам, нежели среднее значение, а значит чуть лучше отражает примерную картину происходящего

# Шаг 4: общий вывод.

В данном проекте мы проверили несколько предположений и убедились, что:
1. количество детей у плательщика влияет на то, выплатит ли он кредит вовремя
2. семейный статус клиента также влияет на то, вернёт ли он деньги банку вовремя
3. уровень дохода клиента влияет на способность выплатить кредит вовремя
4. в зависимости от цели кредита, вероятность выплатить его вовремя также меняется

**Общий вывод.**

В данном проекте мы проверили несколько предположений и убедились, что:

1. **Количество детей** у плательщика **влияет** на то, выплатит ли он кредит вовремя. Посчитав конверсию, мы видим, что клиенты, имеющие 4, 2, 1 детей имеют наибольший шанс не вернуть долг вовремя (конверсии соответственно 9,76% 9,45% 9,23%). У клиентов, имеющих 3 или 0 детей шансов на несвоевременный возврат меньше остальных: 8,18% и 7,54% соответственно; Клиенты, имеющие 5 детей, почти наверное, вернут долг вовремя (нулевая конверсия);


2. **Семейный статус** клиента **также влияет** на то, вернёт ли он деньги банку вовремя.
    * Незамужние (неженатые) плательщики имеют меньший шанс вернуть деньги вовремя (конверсия составила 9,76%).
    * Люди, состоящие в гражданском браке имеют почти такой же шанс (9,31%), но чуть меньше.
    * Клиентам женатым/замужним, разведённым или же вдовам/вдовцам доверять можно больше: шанс, что они вернут долг вовремя, сильно выше (7,56%, 7,06% и 6,62% соответственно).
    
    
3. **Уровень дохода** клиента **влияет** на способность выплатить кредит вовремя.
    * Конверсия по клиентам, имеющим зарплату до 30_000 (коих всего 22) составила 9,09% (**наибольшая конверсия**)
    * Имеющим зарплату в диапазоне [50_001, 200_000] конверсия составила 8,5%
    * Имеющим зарплату свыше 1_000_000 составила 8%
    * Имеющим зарплату в диапозоне [200_001, 1_000_000] составила 7,06%
    * Имеющим зарплату в диапозоне [30_001, 50_000] составила 6,02% (**наименьшая конверсия**)
    
    
4. **В зависимости от цели кредита, вероятность выплатить его вовремя также меняется.** При взятии кредита на операции с автомобилем и получение образования конверсия составила соответственно 9,35% и 9,25% соответственно, а на проведение свадьбы и операции с недвижимостью 7,91% и 7,26% соответственно.

**Здесь конверсия - вероятность того, что тот или иной клиент не закроет кредит вовремя.**